In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Torch Examples

In [66]:
x = Variable(torch.randn(1,3))
y = Variable(torch.randn(1,3))

# Neural Network Examples

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 convolutions
        #kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2x2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        
        # If the size is square, you can only specify single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [85]:
net = Net()

In [86]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0123  0.0908  0.0381 -0.1028 -0.0181  0.0899 -0.0392 -0.0769  0.0525  0.0496
[torch.FloatTensor of size 1x10]



In [87]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [88]:
output = net(input)
target = Variable(torch.arange(1,11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.4242
[torch.FloatTensor of size 1]



In [89]:
print(loss.grad_fn) # MSE Loss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLu

In [90]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  2.8965
 -0.4734
  1.1574
  3.9367
  2.4655
 -0.2138
[torch.FloatTensor of size 6]



In [114]:
import torch.optim as optim

# Create optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# In your training loop
print('conv1.bias before updates')
print(net.conv1.bias)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()

print('conv1.bias before updates')
print(net.conv1.bias)

optimizer.step() # Does update

print('conv1.bias after updates')
print(net.conv1.bias)

conv1.bias before updates
Parameter containing:
-0.0584
-0.0275
 0.0007
 0.1984
 0.0064
-0.0953
[torch.FloatTensor of size 6]

conv1.bias before updates
Parameter containing:
-0.0584
-0.0275
 0.0007
 0.1984
 0.0064
-0.0953
[torch.FloatTensor of size 6]

conv1.bias after updates
Parameter containing:
-0.0497
-0.0227
 0.0033
 0.1989
 0.0095
-0.0939
[torch.FloatTensor of size 6]



In [112]:
optimizer.zero_grad

<bound method Optimizer.zero_grad of <torch.optim.sgd.SGD object at 0x7f376040d5f8>>